In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np

In [ ]:
# 训练集和测试集文件路径
train_path = r'D:\xjtufiles\3ee\DGA\datasets\DGA_data_add_cleaned_train.csv'
test_path = r'D:\xjtufiles\3ee\DGA\datasets\DGA_data_add_cleaned_test.csv'

# 预处理训练集
df_train = pd.read_csv(train_path)

df_train['h2'] = pd.to_numeric(df_train['h2'], errors='coerce')
df_train['ch4'] = pd.to_numeric(df_train['ch4'], errors='coerce')
df_train['c2h6'] = pd.to_numeric(df_train['c2h6'], errors='coerce')
df_train['c2h4'] = pd.to_numeric(df_train['c2h4'], errors='coerce')
df_train['c2h2'] = pd.to_numeric(df_train['c2h2'], errors='coerce')
df_train['act'] = pd.to_numeric(df_train['act'], errors='coerce')

df_train = df_train.reset_index(drop=True)  # 重置索引

# 预处理测试集
df_test = pd.read_csv(test_path)

df_test['h2'] = pd.to_numeric(df_test['h2'], errors='coerce')
df_test['ch4'] = pd.to_numeric(df_test['ch4'], errors='coerce')
df_test['c2h6'] = pd.to_numeric(df_test['c2h6'], errors='coerce')
df_test['c2h4'] = pd.to_numeric(df_test['c2h4'], errors='coerce')
df_test['c2h2'] = pd.to_numeric(df_test['c2h2'], errors='coerce')
df_test['act'] = pd.to_numeric(df_test['act'], errors='coerce')

df_test = df_test.reset_index(drop=True)  # 重置索引

# 现在，df_train 和 df_test 已经分别完成了预处理
print("训练集预处理完成，形状：", df_train.shape)
print("测试集预处理完成，形状：", df_test.shape)

In [ ]:
feature_cols = ['h2', 'ch4', 'c2h6', 'c2h4', 'c2h2']
# 确保特征列为数值类型 (df_train)
for col in feature_cols:
    df_train[col] = pd.to_numeric(df_train[col], errors='coerce')

# 确保目标列为整数类型 (df_train)
df_train['act'] = df_train['act'].astype(int)

print("训练集数据类型：")
print(df_train.dtypes)

# 确保特征列为数值类型 (df_test)
for col in feature_cols:
    df_test[col] = pd.to_numeric(df_test[col], errors='coerce')

# 确保目标列为整数类型 (df_test)
df_test['act'] = df_test['act'].astype(int)

print("\n测试集数据类型：")
print(df_test.dtypes)

In [ ]:
import pandas as pd
import numpy as np

# 假设 X 是一个 Pandas DataFrame，包含 h2, ch4, c2h6, c2h4, c2h2 列

train_all = df_train.copy()
test_all = df_test.copy()

def all_features(X):
    

    R1 = X['c2h2'] / (X['c2h2'] + X['ch4'] + X['c2h6'])
    # R2= X['ch4'] /  (X['c2h2'] + X['ch4'] + X['c2h6'])
    # R3 = X['c2h6'] /  (X['c2h2'] + X['ch4'] + X['c2h6'])
    R4 = X['c2h4'] / (X['c2h6'])
    R5 = X['c2h2'] / (X['c2h4'])
    R6 = X['c2h2'] / (X['h2'] + X['ch4'] + X['c2h2'] + X['c2h4'] + X['c2h6'])
    R7 = X['c2h2'] /  (X['ch4'])
    R8 = (X['c2h2']+X['ch4']) / (X['h2'] )
    # R9 = X['h2'] / (X['c2h2'])
    # R10 = X['c2h2'] / (X['c2h4'] + X['ch4'] + X['c2h6'])
    # R11 = (X['ch4'] + X['c2h6']) / (X['h2'] + X['ch4'] + X['c2h2'] + X['c2h4'] + X['c2h6'])
    # E_H = X['energy_index'] = (X['h2'] * 1 + X['ch4'] * 2 + X['c2h6'] * 3 +  X['c2h4'] * 4 +  X['c2h2'] * 5)
    # # E_L = ( X['h2'] * 1 +  X['ch4'] * 2 + X['c2h6'] * 3)
    E = (X['h2'] * 1 + X['ch4'] * 2 + X['c2h6'] * 3 +  X['c2h4'] * 4 +  X['c2h2'] * 5)
    # 处理异常值
    R1 = R1.replace([np.inf, -np.inf], 0).fillna(0)
    R4 = R4.replace([np.inf, -np.inf], 0).fillna(0)
    R5 = R5.replace([np.inf, -np.inf], 0).fillna(0)
    R6 = R6.replace([np.inf, -np.inf], 0).fillna(0)
    R7 = R7.replace([np.inf, -np.inf], 0).fillna(0)
    R8 = R8.replace([np.inf, -np.inf], 0).fillna(0)
    
    E = E.replace([np.inf, -np.inf], 0).fillna(0)
    # 添加到特征矩阵
    X['R1: C2H2/(C2H2+CH4+C2H6)'] = R1
    X['R4: C2H4/C2H6'] = R4
    X['R5: C2H2/C2H4'] = R5
    X['R6: C2H2/(H2+CH4+C2H2+C2H4+C2H6)'] = R6
    X['R7: C2H2/CH4'] = R7
    X['R8: (C2H2+CH4)/H2'] = R8

    return X

all_train_data = all_features(train_all)
all_test_data = all_features(test_all)


# Gini随机森林 + SMOTE

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (accuracy_score, precision_score, recall_score,
                             f1_score, classification_report, confusion_matrix)
from imblearn.over_sampling import SMOTE, ADASYN, BorderlineSMOTE, KMeansSMOTE

X_train = all_train_data.drop('act', axis=1)
y_train = all_train_data['act']
X_test = all_test_data.drop('act', axis=1)
y_test = all_test_data['act']

# 标签映射（可选）
label_mapping = {
    1: 'HED',
    2: 'HT',
    3: 'LED',
    4: 'LT',
    5: 'MT',
    6: 'PD'
}
target_names = [label_mapping[i] for i in sorted(label_mapping.keys())]

# 修改 train_and_evaluate 函数，返回分类报告和 overall metrics
def train_and_evaluate(X_train, y_train, X_test, y_test):
    model = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=42)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    # 每类指标
    report_dict = classification_report(y_test, y_pred, target_names=target_names, output_dict=True, zero_division=0)
    
    # 转换为DataFrame方便展示
    report_df = pd.DataFrame(report_dict).transpose()

    return report_df

# 存储结果
results = OrderedDict()
results['No Oversampling'] = train_and_evaluate(X_train, y_train, X_test, y_test)

# 只启用 SMOTE（你可以取消注释其他方法）
resamplers = {
    'SMOTE': SMOTE(random_state=42),
    # 'ADASYN': ADASYN(random_state=42),
    # 'BorderlineSMOTE': BorderlineSMOTE(random_state=42),
    # 'KMeansSMOTE': KMeansSMOTE(random_state=42)
}

for name, sampler in resamplers.items():
    X_resampled, y_resampled = sampler.fit_resample(X_train, y_train)
    results[name] = train_and_evaluate(X_resampled, y_resampled, X_test, y_test)

# 输出结果
print("\n=== 每类指标报告 ===")
for method, report_df in results.items():
    print(f"\n[{method}]")
    print(report_df.loc[target_names][['precision', 'recall', 'f1-score']].round(4))


# Gini随机森林 + SMOTE + SHAP

In [ ]:
import shap

# 应用 SMOTE
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train, y_train)

# 训练模型
model = RandomForestClassifier(n_estimators=100, criterion='gini', random_state=42)
model.fit(X_resampled, y_resampled)

# 模型预测并评估
y_pred = model.predict(X_test)
explainer = shap.TreeExplainer(model)
shap_values = explainer.shap_values(X_test)

for i, class_name in enumerate(model.classes_):
    #plt.title(f"SHAP Summary Plot for Class {class_name}", fontsize=15)
    shap.summary_plot(shap_values[:, :, i], X_test, class_names=[class_name], title=f"SHAP Summary Plot for Class {class_name}")
    
    plt.show()
